# 0 环境配置

In [1]:
import os
import glob
import numpy as np
import pandas as pd

# seed
np.random.seed(2019)

from keras.optimizers import *
from keras.utils import multi_gpu_model
import sys
sys.path.append('../')
from dataload.my_image import MyImageDataGenerator
from model.coremodel import incep_resnet_v1_correct as my_model
from model.metric import count_accuracy
from keras.models import load_model
from model.DepthwiseConv2D import DepthwiseConv2D
print("import done")

Using TensorFlow backend.


import done


# 1 预设参数

In [2]:
basedir = "/home/ubuntu/work/Counting/data_spilit/shplan1_link/"
basemodel_name = 'inception_resnet_v2'
model_image_size = (444, 592)
modelname = basemodel_name + '_incep_resnet_v1_correct_shplan1_20181109'
#old_savemodel_path = './inception_v3_model_v9lc_plan6_20180401.h5'
old_savemodel_path = '../savemodel/shplan1_incep_resent_v1_20181107/inception_resnet_v2_incep_resnet_v1_shplan1_20181107_0.0235_0.9556.h5'
savemodel_path = "../savemodel/shplan1_incep_resent_v1_correct_20181109/" + modelname

In [3]:
# define prerpocess_input
from keras.applications.xception import preprocess_input as preprocess_input_ception
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
if basemodel_name == 'resnet50':
    preprocess_input = preprocess_input_resnet
elif basemodel_name == 'xception' or 'inception_v3' or 'inception_resnet_v2':
    preprocess_input = preprocess_input_ception
else:
    print('basemodel_name not defined！')
    raise NameError

# 2 数据生成

In [4]:
train_batch_size = 24
valid_batch_size = 24

In [5]:
train_gen = MyImageDataGenerator(
    preprocessing_function=preprocess_input,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rotation_range=6,
    shear_range=5,
    channel_shift_range=15,
    zoom_range=(.9, 1.1)
)

gen = MyImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.myflow_from_directory(os.path.join(basedir, 'train'),  model_image_size,
                                                shuffle=True, batch_size=train_batch_size, my_horizontal_flip=True,myrescale=True)

valid_generator = gen.myflow_from_directory(os.path.join(basedir, 'valid'),  model_image_size,
                                          shuffle=True, batch_size=valid_batch_size)

Found 12972 images.
Found 1069 images.


In [6]:
train_samples_epoch = train_generator.samples
print("samples_train_epoch = {}".format(train_samples_epoch))

steps_train = len(train_generator)
print("steps_train = {}".format(steps_train))

valid_samples_epoch = valid_generator.samples
print("samples_valid_epoch = {}".format(valid_samples_epoch))

steps_valid = len(valid_generator)
print("steps_valid = {}".format(steps_valid))

samples_train_epoch = 12972
steps_train = 541
samples_valid_epoch = 1069
steps_valid = 45


# 3 搭建模型

In [7]:
model = my_model(basemodel_name, model_image_size)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 444, 592, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 221, 295, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 221, 295, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 221, 295, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [9]:
model.load_weights(old_savemodel_path, by_name=True, skip_mismatch=True)

/home/ubuntu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/keras/engine/topology.py:3242: UserWarning: Skipping loading of weights for layer l_x2_before_dense due to mismatch in shape ((900, 128) vs (600, 128)).
  weight_values[i].shape))


In [10]:
multi_model = multi_gpu_model(model, gpus=2)

# 4 训练模型

In [11]:
from keras.callbacks import Callback, EarlyStopping
from model.loss import myloss

In [12]:
class MyModelCheckpoint(Callback):

    def __init__(self, model, path, init_loss=None):
        self.mymodel = model
        self.path = path
        if init_loss == None:
            self.best_loss = np.inf
        else:
            self.best_loss = init_loss

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs['val_loss']
        val_acc = logs['val__count_accuracy']
        if val_loss < self.best_loss or val_acc>0.91:
            print("\nValidation loss decreased from {0:.5f} to {1:.5f}, saving model".format(self.best_loss, val_loss))
            savedmodel_name = '{}_{:.4f}_{:.4f}.h5'.format(self.path, val_loss, val_acc)
            self.mymodel.save(savedmodel_name, overwrite=True)
            self.best_loss = val_loss

In [13]:
checkpoint = MyModelCheckpoint(model, savemodel_path)

In [14]:
multi_model.compile(optimizer=Adam(lr=1e-3), 
              loss=myloss(),
              metrics=[count_accuracy()])
multi_model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=100,
                   validation_data=valid_generator, validation_steps=steps_valid, 
                          max_queue_size=64, workers=8,
                 callbacks=[checkpoint, 
                EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='min')])

0.001
Epoch 1/100
540/541 [============================>.] - ETA: 1s - loss: 0.0765 - _count_accuracy: 0.5069
Validation loss decreased from inf to 0.03235, saving model
541/541 [==============================] - 732s 1s/step - loss: 0.0764 - _count_accuracy: 0.5072 - val_loss: 0.0324 - val__count_accuracy: 0.6167
Epoch 2/100
541/541 [==============================] - 553s 1s/step - loss: 0.0118 - _count_accuracy: 0.8182 - val_loss: 0.0476 - val__count_accuracy: 0.8398
Epoch 3/100
540/541 [============================>.] - ETA: 0s - loss: 0.0072 - _count_accuracy: 0.8782
Validation loss decreased from 0.03235 to 0.00653, saving model
541/541 [==============================] - 560s 1s/step - loss: 0.0072 - _count_accuracy: 0.8783 - val_loss: 0.0065 - val__count_accuracy: 0.9565
Epoch 4/100
541/541 [==============================] - 556s 1s/step - loss: 0.0049 - _count_accuracy: 0.9087 - val_loss: 0.0092 - val__count_accuracy: 0.8991
Epoch 5/100
540/541 [============================>.] -

In [15]:

multi_model.compile(optimizer=Adam(lr=1e-4), 
              loss=myloss(),
              metrics=[count_accuracy()])
multi_model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=100,
                   validation_data=valid_generator, validation_steps=steps_valid, 
                          max_queue_size=64, workers=8,
                 callbacks=[checkpoint, 
                EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='min')])

1e-04
Epoch 1/100
540/541 [============================>.] - ETA: 1s - loss: 0.0019 - _count_accuracy: 0.9619
Validation loss decreased from 0.03094 to 0.00897, saving model
541/541 [==============================] - 593s 1s/step - loss: 0.0019 - _count_accuracy: 0.9619 - val_loss: 0.0090 - val__count_accuracy: 0.9583
Epoch 2/100
540/541 [============================>.] - ETA: 0s - loss: 0.0013 - _count_accuracy: 0.9748
Validation loss decreased from 0.00897 to 0.00905, saving model
541/541 [==============================] - 556s 1s/step - loss: 0.0013 - _count_accuracy: 0.9747 - val_loss: 0.0090 - val__count_accuracy: 0.9583
Epoch 3/100
540/541 [============================>.] - ETA: 0s - loss: 0.0015 - _count_accuracy: 0.9772
Validation loss decreased from 0.00905 to 0.00701, saving model
541/541 [==============================] - 556s 1s/step - loss: 0.0015 - _count_accuracy: 0.9772 - val_loss: 0.0070 - val__count_accuracy: 0.9556
Epoch 4/100
540/541 [============================>.]

In [16]:
multi_model.compile(optimizer=Adam(lr=1e-5), 
              loss=myloss(),
              metrics=[count_accuracy()])
multi_model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=100,
                   validation_data=valid_generator, validation_steps=steps_valid, 
                          max_queue_size=64, workers=8,
                 callbacks=[checkpoint, 
                EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=0, mode='min')])

1e-05
Epoch 1/100
540/541 [============================>.] - ETA: 1s - loss: 6.1694e-04 - _count_accuracy: 0.9889
Validation loss decreased from 0.00676 to 0.00729, saving model
541/541 [==============================] - 598s 1s/step - loss: 6.1797e-04 - _count_accuracy: 0.9888 - val_loss: 0.0073 - val__count_accuracy: 0.9546
Epoch 2/100
540/541 [============================>.] - ETA: 1s - loss: 5.8294e-04 - _count_accuracy: 0.9910
Validation loss decreased from 0.00729 to 0.00615, saving model
541/541 [==============================] - 563s 1s/step - loss: 5.8205e-04 - _count_accuracy: 0.9911 - val_loss: 0.0061 - val__count_accuracy: 0.9593
Epoch 3/100
540/541 [============================>.] - ETA: 1s - loss: 5.3064e-04 - _count_accuracy: 0.9910
Validation loss decreased from 0.00615 to 0.00634, saving model
541/541 [==============================] - 561s 1s/step - loss: 5.2981e-04 - _count_accuracy: 0.9910 - val_loss: 0.0063 - val__count_accuracy: 0.9583
Epoch 4/100
540/541 [=======

# 5 测试模型